## Fashion MNIST 데이터셋을 이용한 합성곱 신경망(CNN) 구현

이 노트북은 Keras를 사용하여 Fashion MNIST 데이터셋을 분류하는 간단한 합성곱 신경망(CNN) 모델을 구축, 훈련 및 평가하는 과정을 보여줍니다.

### 1. 데이터 로드 및 확인
Keras에 내장된 Fashion MNIST 데이터셋을 로드하고, 훈련 데이터와 테스트 데이터의 형태(shape)를 출력하여 구조를 확인합니다.

In [ ]:
from keras.datasets import fashion_mnist

(X_train, y_train), (X_test, y_test) = fashion_mnist.load_data()

print("X_train shape:", X_train.shape)
print("y_train shape:", y_train.shape)
print("X_test shape:", X_test.shape)
print("y_test shape:", y_test.shape)

### 2. CNN 모델 정의
합성곱 신경망 모델을 `Sequential` API를 사용하여 구성합니다.

모델 구조:
1. **Rescaling Layer**: 이미지 픽셀 값을 [0, 255] 범위에서 [0, 1] 범위로 정규화합니다.
2. **Conv2D Layer (1)**: 32개의 (3,3) 필터와 ReLU 활성화 함수를 사용하는 첫 번째 합성곱 층입니다.
3. **Conv2D Layer (2)**: 64개의 (3,3) 필터와 ReLU 활성화 함수를 사용하는 두 번째 합성곱 층입니다.
4. **MaxPooling2D Layer**: (2,2) 크기의 풀링을 통해 특성 맵의 크기를 줄여 과대적합을 방지합니다.
5. **Flatten Layer**: 4차원 특성 맵을 2차원 벡터로 변환하여 완전 연결망에 입력할 수 있도록 합니다.
6. **Dense Layer (1)**: 128개의 뉴런과 ReLU 활성화 함수를 갖는 완전 연결 층입니다.
7. **Dense Layer (2)**: 64개의 뉴런과 ReLU 활성화 함수를 갖는 완전 연결 층입니다.
8. **Output Layer**: 10개의 클래스를 분류하기 위해 10개의 뉴런과 Softmax 활성화 함수를 사용하는 출력층입니다.

In [ ]:
from keras import models, layers

img_height = 28
img_width = 28

network = models.Sequential([
    # 스케일링은 반드시, 내부 스케일링 사용
    layers.Rescaling(1./255, input_shape=(img_height, img_width, 1)),
    
    # 합성곱 층
    layers.Conv2D(32, (3, 3), activation='relu'), # 32 - 출력 필터 수, (3,3) - 필터 크기
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)), # 서브샘플링, 특성 개수를 줄여 과대적합 방지
    
    # CNN과 완전연결망을 연결하기 위한 Flatten 층 (4D -> 2D)
    layers.Flatten(),
    
    # 완전연결망
    layers.Dense(128, activation='relu'),
    layers.Dense(64, activation='relu'),
    layers.Dense(10, activation='softmax') # 마지막 출력층
])

### 3. 모델 컴파일
훈련 프로세스를 설정합니다.
- **Optimizer**: `adam` 최적화 알고리즘을 사용합니다.
- **Loss Function**: 레이블이 정수 형태이므로 `SparseCategoricalCrossentropy`를 손실 함수로 사용합니다.
- **Metrics**: 훈련 및 평가 과정에서 `accuracy`(정확도)를 모니터링합니다.

In [ ]:
import tensorflow as tf
import keras

network.compile(optimizer='adam',
                loss=keras.losses.SparseCategoricalCrossentropy(), # 라벨이 정수형일 때 사용
                metrics=['accuracy'])

### 4. 모델 요약 확인
 `summary()` 메서드를 통해 모델의 전체 구조와 각 층의 파라미터 수를 확인합니다.

In [ ]:
network.summary()

### 5. 모델 훈련
`fit()` 메서드를 사용하여 모델을 훈련시킵니다.
- `epochs=10`: 전체 훈련 데이터셋을 10번 반복하여 학습합니다.
- `validation_split=0.2`: 훈련 데이터의 20%를 검증 데이터로 사용하여 각 에포크마다 모델 성능을 평가합니다.

In [ ]:
history = network.fit(X_train, y_train, epochs=10, validation_split=0.2)

### 6. 모델 평가
`evaluate()` 메서드를 사용하여 훈련된 모델의 성능을 훈련 데이터셋과 테스트 데이터셋에 대해 각각 평가하고, 손실(loss)과 정확도(accuracy)를 출력합니다.

In [ ]:
print("\n--- Evaluation ---")
train_loss, train_acc = network.evaluate(X_train, y_train)
print(f"훈련셋 평가: Loss={train_loss:.4f}, Accuracy={train_acc:.4f}")

test_loss, test_acc = network.evaluate(X_test, y_test)
print(f"테스트셋 평가: Loss={test_loss:.4f}, Accuracy={test_acc:.4f}")